ClinVar!

In [1]:
import ndjson
import pandas as pd
import numpy as np

In [2]:
import logging
from enum import Enum
import re
import csv
from pathlib import Path
import zipfile

from dotenv import load_dotenv

#from variation.query import QueryHandler

logging.getLogger("root").setLevel(logging.WARNING)

In [3]:
file = open('/Users/rsaxs014/Downloads/output-variation_identity.ndjson')
records = ndjson.load(file)

df = pd.json_normalize(records)

df

,in.id,in.name,in.subclass_type,in.variation_type,in.xrefs,in.vrs_xform_plan.type,in.vrs_xform_plan.inputs,in.vrs_xform_plan.policy,out.id,out.type,...,out.state.seq_expr.location.id,out.state.seq_expr.location.type,out.state.seq_expr.location.sequence_id,out.state.seq_expr.location.start.type,out.state.seq_expr.location.start.value,out.state.seq_expr.location.end.type,out.state.seq_expr.location.end.value,out.state.seq_expr.reverse_complement,out.state.count.type,out.state.count.value
0,16098,"SLC2A2, 1-BP INS, 793C",SimpleAllele,Insertion,OMIM:138160.0009,Text,[id],No hgvs or location info,Text:clinvar:16098,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,425693,NM_001204.6(BMPR2):c.77-?_247+?dup,SimpleAllele,Duplication,NaN,Text,[id],No hgvs or location info,Text:clinvar:425693,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,90650,NM_000251.2(MSH2):c.1387-?_1510+?del,SimpleAllele,Deletion,NaN,Text,[id],No hgvs or location info,Text:clinvar:90650,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2446408,"CDHR1, 783G-A (rs147346345)",SimpleAllele,single nucleotide variant,OMIM:609502.0005,Text,[id],No hgvs or location info,Text:clinvar:2446408,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14905,"HLA-DRB1, HLA-DRB1*1101",SimpleAllele,Variation,OMIM:142857.0001,Text,[id],No hgvs or location info,Text:clinvar:14905,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2210622,1464440,NM_001267550.2(TTN):c.29512_29513insGGCCGGGCGC...,SimpleAllele,Insertion,NaN,Text,[id],Invalid/unsupported hgvs,Text:clinvar:1464440,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2210623,1453116,NM_000051.4(ATM):c.3376_3377insGGCCGGGCGCGGTGG...,SimpleAllele,Insertion,NaN,Text,[id],Invalid/unsupported hgvs,Text:clinvar:1453116,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2210624,1181160,NM_030962.4(SBF2):c.55+97_55+98insCGGGCGTCGGGGC,SimpleAllele,Microsatellite,NaN,Allele,[canonical_spdi],Canonical SPDI,ga4gh:VA.x2E2GJQA5ovZFJ1SKxg3oHzkUXv5ZGfv,Allele,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2210625,2202105,NM_153676.4(USH1C):c.496+14_496+15insGTACTCCAT...,SimpleAllele,Microsatellite,NaN,Allele,[canonical_spdi],Canonical SPDI,ga4gh:VA.5jQEJZIbf1ttjbIGJVhfrHR9XcB7vqWD,Allele,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Add Supported Status- based on out.type

In [4]:
df['out.type'] = df['out.type'].fillna("None")
df

,in.id,in.name,in.subclass_type,in.variation_type,in.xrefs,in.vrs_xform_plan.type,in.vrs_xform_plan.inputs,in.vrs_xform_plan.policy,out.id,out.type,...,out.state.seq_expr.location.id,out.state.seq_expr.location.type,out.state.seq_expr.location.sequence_id,out.state.seq_expr.location.start.type,out.state.seq_expr.location.start.value,out.state.seq_expr.location.end.type,out.state.seq_expr.location.end.value,out.state.seq_expr.reverse_complement,out.state.count.type,out.state.count.value
0,16098,"SLC2A2, 1-BP INS, 793C",SimpleAllele,Insertion,OMIM:138160.0009,Text,[id],No hgvs or location info,Text:clinvar:16098,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,425693,NM_001204.6(BMPR2):c.77-?_247+?dup,SimpleAllele,Duplication,NaN,Text,[id],No hgvs or location info,Text:clinvar:425693,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,90650,NM_000251.2(MSH2):c.1387-?_1510+?del,SimpleAllele,Deletion,NaN,Text,[id],No hgvs or location info,Text:clinvar:90650,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2446408,"CDHR1, 783G-A (rs147346345)",SimpleAllele,single nucleotide variant,OMIM:609502.0005,Text,[id],No hgvs or location info,Text:clinvar:2446408,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14905,"HLA-DRB1, HLA-DRB1*1101",SimpleAllele,Variation,OMIM:142857.0001,Text,[id],No hgvs or location info,Text:clinvar:14905,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2210622,1464440,NM_001267550.2(TTN):c.29512_29513insGGCCGGGCGC...,SimpleAllele,Insertion,NaN,Text,[id],Invalid/unsupported hgvs,Text:clinvar:1464440,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2210623,1453116,NM_000051.4(ATM):c.3376_3377insGGCCGGGCGCGGTGG...,SimpleAllele,Insertion,NaN,Text,[id],Invalid/unsupported hgvs,Text:clinvar:1453116,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2210624,1181160,NM_030962.4(SBF2):c.55+97_55+98insCGGGCGTCGGGGC,SimpleAllele,Microsatellite,NaN,Allele,[canonical_spdi],Canonical SPDI,ga4gh:VA.x2E2GJQA5ovZFJ1SKxg3oHzkUXv5ZGfv,Allele,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2210625,2202105,NM_153676.4(USH1C):c.496+14_496+15insGTACTCCAT...,SimpleAllele,Microsatellite,NaN,Allele,[canonical_spdi],Canonical SPDI,ga4gh:VA.5jQEJZIbf1ttjbIGJVhfrHR9XcB7vqWD,Allele,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df["out.type"].value_counts()

out.type
Allele              2113773
Text                  72105
CopyNumberChange      12424
None                   7445
CopyNumberCount        4880
Name: count, dtype: int64

In [6]:
df["support_status"] = df["out.type"].copy()

df.loc[df["support_status"] == "Allele", "support_status"] = True
df.loc[df["support_status"] == "Text", "support_status"] = False
df.loc[df["support_status"] == "CopyNumberChange", "support_status"] = True
df.loc[df["support_status"] == "CopyNumberCount", "support_status"] = True
df.loc[df["support_status"] == "None", "support_status"] = False

df

,in.id,in.name,in.subclass_type,in.variation_type,in.xrefs,in.vrs_xform_plan.type,in.vrs_xform_plan.inputs,in.vrs_xform_plan.policy,out.id,out.type,...,out.state.seq_expr.location.type,out.state.seq_expr.location.sequence_id,out.state.seq_expr.location.start.type,out.state.seq_expr.location.start.value,out.state.seq_expr.location.end.type,out.state.seq_expr.location.end.value,out.state.seq_expr.reverse_complement,out.state.count.type,out.state.count.value,support_status
0,16098,"SLC2A2, 1-BP INS, 793C",SimpleAllele,Insertion,OMIM:138160.0009,Text,[id],No hgvs or location info,Text:clinvar:16098,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,425693,NM_001204.6(BMPR2):c.77-?_247+?dup,SimpleAllele,Duplication,NaN,Text,[id],No hgvs or location info,Text:clinvar:425693,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,90650,NM_000251.2(MSH2):c.1387-?_1510+?del,SimpleAllele,Deletion,NaN,Text,[id],No hgvs or location info,Text:clinvar:90650,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,2446408,"CDHR1, 783G-A (rs147346345)",SimpleAllele,single nucleotide variant,OMIM:609502.0005,Text,[id],No hgvs or location info,Text:clinvar:2446408,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,14905,"HLA-DRB1, HLA-DRB1*1101",SimpleAllele,Variation,OMIM:142857.0001,Text,[id],No hgvs or location info,Text:clinvar:14905,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2210622,1464440,NM_001267550.2(TTN):c.29512_29513insGGCCGGGCGC...,SimpleAllele,Insertion,NaN,Text,[id],Invalid/unsupported hgvs,Text:clinvar:1464440,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2210623,1453116,NM_000051.4(ATM):c.3376_3377insGGCCGGGCGCGGTGG...,SimpleAllele,Insertion,NaN,Text,[id],Invalid/unsupported hgvs,Text:clinvar:1453116,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2210624,1181160,NM_030962.4(SBF2):c.55+97_55+98insCGGGCGTCGGGGC,SimpleAllele,Microsatellite,NaN,Allele,[canonical_spdi],Canonical SPDI,ga4gh:VA.x2E2GJQA5ovZFJ1SKxg3oHzkUXv5ZGfv,Allele,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2210625,2202105,NM_153676.4(USH1C):c.496+14_496+15insGTACTCCAT...,SimpleAllele,Microsatellite,NaN,Allele,[canonical_spdi],Canonical SPDI,ga4gh:VA.5jQEJZIbf1ttjbIGJVhfrHR9XcB7vqWD,Allele,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [7]:
df['support_status'].value_counts()

support_status
True     2131077
False      79550
Name: count, dtype: int64

## Add Normalization Status- based on out.errors

In [8]:
### errors stored as a list of values, some of which are strings and other of which are dictionaries 
### (determined by whether error was handled at the level of Variation Normalizer or after the normalizer)
### this function extracts the text error responses for better readability and ease string processing

def get_errors(errors):
    errors_out = []
    for e in errors:
        if type(e)== str:
            errors_out.append(e)
        elif type(e)==dict:
            for k,v in e.items():
                if type(v) == str:
                    errors_out.append('[normalizer] ' + str(v))
                elif type(v) == list:
                    errors_out.append('[normalizer] ' + ';'.join(v))
                #errors_out.append('[NORMALIZER] '+';'.join(e.get('response-errors',[])))
    return ';'.join(errors_out)

In [9]:
### apply the error extraction function to our dataframe's column containing the errors
df['error_string'] = df['out.errors'].fillna('').apply(get_errors)

In [10]:
### this is the number of unique error strings
### there are many different strings because many of the errors contain specific genomic coordinates, which are unlikely to occur more than once
df['error_string'].nunique()

63108

In [11]:
### to get the core error message, we'll simply strip all the numeric values out and replace them with "#"
def reduce_errors(error_string):
    out = error_string.lower()
    out = re.sub('\d+','#',out)
    return out

In [12]:
df['error_string_reduce'] = df['error_string'].apply(reduce_errors)

In [13]:
df['error_string_reduce'].value_counts()

error_string_reduce
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         2141287
variation record was too long to normalize (#), treating as text                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [14]:
df['error_string_reduce'] = df['error_string'].apply(reduce_errors).replace('',np.nan)

In [15]:
df['error_string_reduce'].value_counts()

error_string_reduce
variation record was too long to normalize (#), treating as text                                                                                                                                                                                                                                                                                                                                                                                                                                         61464
[normalizer] clinvar-gk-pilot.main/validate-response;[normalizer] {"warnings":["copy_number_count mode requires `baseline_copies`"],"service_meta_":{"version":"#.#.#-dev#","response_datetime":"#-#-#t#:#:#.#","name":"variation-normalizer","url":"https://github.com/cancervariants/variation-normalization"},"hgvs_expr":"nc_#.#:g.(?_#)_(#_?)del","copy_number_count":null};[normalizer] error returned from variation normalizer;[normalizer] copy_number_count mode requires `baseline_copies`

In [16]:
mycols = df["out.errors"]

In [17]:
df["normalize_status"] =  df["out.errors"].isna()
df

,in.id,in.name,in.subclass_type,in.variation_type,in.xrefs,in.vrs_xform_plan.type,in.vrs_xform_plan.inputs,in.vrs_xform_plan.policy,out.id,out.type,...,out.state.seq_expr.location.start.value,out.state.seq_expr.location.end.type,out.state.seq_expr.location.end.value,out.state.seq_expr.reverse_complement,out.state.count.type,out.state.count.value,support_status,error_string,error_string_reduce,normalize_status
0,16098,"SLC2A2, 1-BP INS, 793C",SimpleAllele,Insertion,OMIM:138160.0009,Text,[id],No hgvs or location info,Text:clinvar:16098,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,False,,NaN,True
1,425693,NM_001204.6(BMPR2):c.77-?_247+?dup,SimpleAllele,Duplication,NaN,Text,[id],No hgvs or location info,Text:clinvar:425693,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,False,,NaN,True
2,90650,NM_000251.2(MSH2):c.1387-?_1510+?del,SimpleAllele,Deletion,NaN,Text,[id],No hgvs or location info,Text:clinvar:90650,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,False,,NaN,True
3,2446408,"CDHR1, 783G-A (rs147346345)",SimpleAllele,single nucleotide variant,OMIM:609502.0005,Text,[id],No hgvs or location info,Text:clinvar:2446408,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,False,,NaN,True
4,14905,"HLA-DRB1, HLA-DRB1*1101",SimpleAllele,Variation,OMIM:142857.0001,Text,[id],No hgvs or location info,Text:clinvar:14905,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,False,,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2210622,1464440,NM_001267550.2(TTN):c.29512_29513insGGCCGGGCGC...,SimpleAllele,Insertion,NaN,Text,[id],Invalid/unsupported hgvs,Text:clinvar:1464440,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,False,,NaN,True
2210623,1453116,NM_000051.4(ATM):c.3376_3377insGGCCGGGCGCGGTGG...,SimpleAllele,Insertion,NaN,Text,[id],Invalid/unsupported hgvs,Text:clinvar:1453116,Text,...,NaN,NaN,NaN,NaN,NaN,NaN,False,,NaN,True
2210624,1181160,NM_030962.4(SBF2):c.55+97_55+98insCGGGCGTCGGGGC,SimpleAllele,Microsatellite,NaN,Allele,[canonical_spdi],Canonical SPDI,ga4gh:VA.x2E2GJQA5ovZFJ1SKxg3oHzkUXv5ZGfv,Allele,...,NaN,NaN,NaN,NaN,NaN,NaN,True,,NaN,True
2210625,2202105,NM_153676.4(USH1C):c.496+14_496+15insGTACTCCAT...,SimpleAllele,Microsatellite,NaN,Allele,[canonical_spdi],Canonical SPDI,ga4gh:VA.5jQEJZIbf1ttjbIGJVhfrHR9XcB7vqWD,Allele,...,NaN,NaN,NaN,NaN,NaN,NaN,True,,NaN,True


In [18]:
df["normalize_status"].value_counts()

normalize_status
True     2141287
False      69340
Name: count, dtype: int64

## Create groups of variants based on Supported and Normalized Status

In [28]:
supported_df= df.copy()
# df_supported=df_supported.loc[(df_supported['support_status'] == 'True') & 
#                          (df_supported['normalize_status'] == 'True')]
# #df_supported=df_supported[df_supported["support_status"] ==True ] & df_supported[df_supported["normalize_status"] == True]
# #df_supported= df_supported.where(df_supported['support_status']==True) & (df_supported['normalize_status']==True)
# df_supported

In [30]:
supported_df=supported_df.loc[(supported_df['support_status'] == True) & 
                         (supported_df['normalize_status'] == True)]
supported_df

,in.id,in.name,in.subclass_type,in.variation_type,in.xrefs,in.vrs_xform_plan.type,in.vrs_xform_plan.inputs,in.vrs_xform_plan.policy,out.id,out.type,...,out.state.seq_expr.location.start.value,out.state.seq_expr.location.end.type,out.state.seq_expr.location.end.value,out.state.seq_expr.reverse_complement,out.state.count.type,out.state.count.value,support_status,error_string,error_string_reduce,normalize_status
149,1676330,NM_054027.6(ANKH):c.259G>A (p.Val87Ile),SimpleAllele,single nucleotide variant,NaN,Allele,[hgvs],Remaining valid hgvs alleles,ga4gh:VA.T3hLVZajyx7AGQKV7la2RgE9CO0Wiv2y,Allele,...,NaN,NaN,NaN,NaN,NaN,NaN,True,,NaN,True
150,1676476,NM_012309.5(SHANK2):c.460C>T (p.Gln154Ter),SimpleAllele,single nucleotide variant,NaN,Allele,[hgvs],Remaining valid hgvs alleles,ga4gh:VA.dZAOK2Sy6aJlXan36nZYUjEJpk5WX8pF,Allele,...,NaN,NaN,NaN,NaN,NaN,NaN,True,,NaN,True
151,1676638,NM_000094.4(COL7A1):c.8729G>T (p.Gly2910Val),SimpleAllele,single nucleotide variant,NaN,Allele,[hgvs],Remaining valid hgvs alleles,ga4gh:VA.aUAN4LCHVZ_CPwAQfkVuX42g_Gi6WlBu,Allele,...,NaN,NaN,NaN,NaN,NaN,NaN,True,,NaN,True
152,1676567,NM_005445.4(SMC3):c.1343dup (p.Glu449fs),SimpleAllele,Duplication,NaN,CopyNumberChange,[hgvs],Copy number change (cn loss|del and cn gain|dup),ga4gh:CX.-9T00EYMyHQKC7DVOenHi-QH8aIOcc0D,CopyNumberChange,...,NaN,NaN,NaN,NaN,NaN,NaN,True,,NaN,True
153,1325687,NM_000059.4(BRCA2):c.3692C>G (p.Thr1231Ser),SimpleAllele,single nucleotide variant,NaN,Allele,[hgvs],Remaining valid hgvs alleles,ga4gh:VA.MLNaXFddWSZRL0vRCHJceHBF09R_JgBt,Allele,...,NaN,NaN,NaN,NaN,NaN,NaN,True,,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2210620,1496502,NM_007255.3(B4GALT7):c.881_882insTGAGGTGGATTAA...,SimpleAllele,Insertion,NaN,Allele,[canonical_spdi],Canonical SPDI,ga4gh:VA.P-rdDkBZMl4kwW-ZHmsMOy0ik5MKa4LI,Allele,...,NaN,NaN,NaN,NaN,NaN,NaN,True,,NaN,True
2210621,715720,NM_033026.6(PCLO):c.3300+7_3300+8insTTTATATATA...,SimpleAllele,Insertion,dbSNP:746054139,Allele,[canonical_spdi],Canonical SPDI,ga4gh:VA._iakrjaN6VkKr1AGuegIUY4vKZxnQVkT,Allele,...,NaN,NaN,NaN,NaN,NaN,NaN,True,,NaN,True
2210624,1181160,NM_030962.4(SBF2):c.55+97_55+98insCGGGCGTCGGGGC,SimpleAllele,Microsatellite,NaN,Allele,[canonical_spdi],Canonical SPDI,ga4gh:VA.x2E2GJQA5ovZFJ1SKxg3oHzkUXv5ZGfv,Allele,...,NaN,NaN,NaN,NaN,NaN,NaN,True,,NaN,True
2210625,2202105,NM_153676.4(USH1C):c.496+14_496+15insGTACTCCAT...,SimpleAllele,Microsatellite,NaN,Allele,[canonical_spdi],Canonical SPDI,ga4gh:VA.5jQEJZIbf1ttjbIGJVhfrHR9XcB7vqWD,Allele,...,NaN,NaN,NaN,NaN,NaN,NaN,True,,NaN,True


In [31]:
supported_not_normalized_df =  df.copy()

In [32]:
supported_not_normalized_df = supported_not_normalized_df.loc[(supported_not_normalized_df['support_status'] == True) & 
                         (supported_not_normalized_df['normalize_status'] == False)]
supported_not_normalized_df

,in.id,in.name,in.subclass_type,in.variation_type,in.xrefs,in.vrs_xform_plan.type,in.vrs_xform_plan.inputs,in.vrs_xform_plan.policy,out.id,out.type,...,out.state.seq_expr.location.start.value,out.state.seq_expr.location.end.type,out.state.seq_expr.location.end.value,out.state.seq_expr.reverse_complement,out.state.count.type,out.state.count.value,support_status,error_string,error_string_reduce,normalize_status
